[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ai-forever/gigachain/blob/master/docs/extras/use_cases/summarization.ipynb)

## Варианты использования

Предположим, у вас есть набор документов (PDF-файлы, страницы Notion, вопросы клиентов и т. д.), и вы хотите обобщить их содержимое.

LLM являются отличным инструментом для этого, учитывая их навыки понимания и синтеза текста.

В этом пошаговом руководстве мы рассмотрим, как выполнить обобщение документов с помощью LLM.

![Use case 1](../../static/img/summarization_use_case_1.png)

## Обзор

Основной вопрос при создании сумматора — как передать ваши документы в контекстное окно LLM. Два распространенных подхода к этому:

1.  `Stuff`: просто "сложите" все свои документы в один промпт. Это самый простой подход, который используется для документов, которые можно поместить целиком в контекст запроса.

2. `Map-reduce`: Суммаризирует каждый документ по-отдельности на шаге map, затем суммаризирует все результаты на шаге reduce для получения финального саммари.

3. `Refine`: Суммаризирует часть документов, затем добавляет новые документы к полученной суммаризации и снова суммаризирует.

4. `Conditional`: Суммаризует документы исходя из длины документы, либо методом Stuff, либо Map-reduce

![Use case 2](../../static/img/summarization_use_case_2.png)

 При использоваии параметра `chain_type="stuff"` следует выбирать LLM с большим размером контекстного окна, например:

* 16k token OpenAI `gpt-3.5-turbo-16k` 
* 100k token Anthropic [Claude-2](https://www.anthropic.com/index/claude-2)

Также можно использовать другие типы суммаризации `chain_type="map_reduce"` или `chain_type="refine"` (подробнее смомтрите раздел [Option 3. Refine](/docs/docs/use_cases/summarization.ipynb)).

In [1]:
with open("credentials.txt", 'r', encoding='utf-8') as file:
    credentials = file.read()

## Подход 2. Map-Reduce

Рассмотрим пример суммаризации на основании подхода map-reduce для статей, загружаемых из Википедии.

Статья из Википедии загружается целиком с помощью `WikipediaLoader`, после чего с помощью `CharacterTextSplitter` делится на части размером около 5000 символов.

In [28]:
from langchain.chains.summarize import load_summarize_chain
from langchain.chat_models.gigachat import GigaChat
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

# loader = TextLoader("assets/habr_paper.txt")
# docs = loader.load()

loader = WebBaseLoader("https://habr.com/ru/companies/sberdevices/articles/794773/")
docs = loader.load()

split_docs = CharacterTextSplitter(chunk_size=5000, chunk_overlap=500).split_documents(
    docs
)
print(f"Parts count: {len(split_docs)}")

Created a chunk of size 9666, which is longer than the specified 5000


Parts count: 4


In [34]:
print(docs[0].metadata['description'])

Всем привет! На связи лид разработки SDK GigaChat’a — Константин Крестников. В этой статье я расскажу о том, что такое GigaChain и как в целом SDK позволяет упростить жизнь разработчика LLM, например,...


In [7]:
from langchain_community.document_loaders import ArxivLoader

query_params = {
    'start': 0,  # retreive the first 5 results
    'max_results': 5,
    'sortBy': "submittedDate",
    'sortOrder': "descending"
}

docs = ArxivLoader(query="Attention is all you need", load_max_docs=2).load()
len(docs)

2

In [5]:
# Write data to .txt file
with open("assets/output.txt", 'w', encoding='utf-8') as file:
    file.write(str(docs[0].page_content))

Далее с помощью `GigaChat`` выполняется суммаризация каждой части (фаза map). В конце все суммаризированные части объединяются в одну, после чего выполняется суммарная суммаризация.

In [18]:
giga = GigaChat(credentials=credentials, scope="GIGACHAT_API_CORP", verify_ssl_certs=False)
chain = load_summarize_chain(giga, chain_type="map_reduce")
res = chain.run(split_docs)

print("\n\n===")
print(res)

Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist
Giga generation stopped with reason: blacklist


ResponseError: (URL('https://gigachat.devices.sberbank.ru/api/v1/chat/completions'), 413, b'<html>\r\n<head><title>413 Request Entity Too Large</title></head>\r\n<body>\r\n<center><h1>413 Request Entity Too Large</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n', Headers({'server': 'nginx', 'date': 'Thu, 25 Apr 2024 05:01:53 GMT', 'content-type': 'text/html', 'content-length': '176', 'connection': 'close', 'allow': 'GET, POST', 'strict-transport-security': 'max-age=31536000; includeSubDomains'}))

### Погружаемся глубже
 
**Кастомизация** 

* Вы можете самостоятельно настроить промпты для каждой фазы map-reduce.

**Реальные примеры использования**

* См. [this blog post](https://blog.langchain.dev/llms-to-improve-documentation/) кейс по анализу взаимодействия с пользователем (вопросы по документации LangChain)!  
* Данном посте [repo](https://github.com/mendableai/QA_clustering) представлена кластеризация как средство суммаризации.
* Это открывает третий путь, помимо подходов «stuff» или «map-reduce», который стоит рассмотреть.

![Image description](../../static/img/summarization_use_case_3.png)

## Подход 3. Refine
 
[Refine](/docs/modules/chains/document/refine) подход похож на mad-reduce:

> Цепочка refine создает ответ, перебирая входные документы и итеративно обновляя свой ответ. В цикле происходит суммаризация каждого документа с добавлением полученной на предыдущем шаге суммаризации предыдущих документов.

Вы можете легко запустить эту цепочку, использовав параметр `chain_type="refine"`.

In [10]:
chain = load_summarize_chain(giga, chain_type="refine")
chain.run(split_docs)

'GigaChain - это форк открытой библиотеки LangChain на Python, который был создан для упрощения жизни разработчика. Он содержит множество компонентов, позволяющих работать с промптами, объединять вызовы к большим языковым моделям в цепочки, загружать данные из разных источников и сохранять обработанные ответы языковой модели. GigaChain отличается от LangChain тем, что он переведен на русский язык, адаптирован к экосистеме русскоязычных языковых моделей и, конечно же, к GigaChat. Фреймворк совместим со сторонним сервисом LangSmith - платформой для разработчиков, которая позволяет отлаживать, тестировать, оценивать и отслеживать цепочки, построенные на любой платформе LLM. GigaChain содержит большое количество инструментов и примеров, которые можно использовать в качестве основы своего проекта или просто изучать для расширения кругозора в области возможных кейсов применений LLM.\n\nRAG (Retrieval Augmented Generation) - это подход к обогащению вопроса пользователя дополнительными данными

It's also possible to supply a prompt and return intermediate steps.

In [11]:
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary in Italian"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=giga,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    return_intermediate_steps=True,
    input_key="input_documents",
    output_key="output_text",
)
result = chain({"input_documents": split_docs}, return_only_outputs=True)

/workspace/Syrenny/GigaHack/Syrenny/giga_venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Giga generation stopped with reason: length


In [12]:
print(result["output_text"])

Le nostre opportunità sono di raffinare il sommario esistente con un po' più di contesto sottostante.


In [13]:
print("\n\n".join(result["intermediate_steps"][:3]))

GigaChain - это форк открытой библиотеки LangChain на Python, который был создан для упрощения жизни разработчика. Он содержит множество компонентов, позволяющих работать с промптами, объединять вызовы к большим языковым моделям в цепочки, загружать данные из разных источников и сохранять обработанные ответы языковой модели. GigaChain отличается от LangChain тем, что он переведен на русский язык, адаптирован к экосистеме русскоязычных языковых моделей и, конечно же, к GigaChat. Фреймворк совместим со сторонним сервисом LangSmith - платформой для разработчиков, которая позволяет отлаживать, тестировать, оценивать и отслеживать цепочки, построенные на любой платформе LLM.

La nostra opportunità è di raffinare il sommario esistente con un po' più di contesto sottostante.

------------
In questi casi, uno dei metodi è RAG (Retrieval Augmented Generation), che spiegherò in seguito.

Risposte alle domande dai documenti (RAG)
L'idea di RAG è arricchire la domanda del utente con informazioni a

В случае ниже статья не превышает 1000 токенов, поэтому мы суммаризуем её методом Stuff

В случае ниже, у нас документы превышают 1000 токенов, поэтому мы используем Map-Reduce